## Pasos

In [1]:
# Libraries

#from ibapi.client import EClient
#from ibapi.wrapper import EWrapper
#from ibapi.contract import Contract


from ib_insync import IB, Forex, Stock, MarketOrder
from ib_insync import *


import threading
import time
import asyncio
import pandas as pd
import numpy as np
import time



#### 1 Conneciton to Ibapi

In [3]:
util.startLoop()
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=9)

<IB connected to 127.0.0.1:7497 clientId=9>

In [4]:
# Account possitions
ib.positions()

[Position(account='DU6818917', contract=Stock(conId=107113386, symbol='META', exchange='NASDAQ', currency='USD', localSymbol='META', tradingClass='NMS'), position=50.0, avgCost=186.38)]

In [5]:
# Account info
[v for v in ib.accountValues() if v.tag == 'NetLiquidationByCurrency' and v.currency == 'BASE']

[AccountValue(account='DU6818917', tag='NetLiquidationByCurrency', value='1001611', currency='BASE', modelCode='')]

In [6]:
ib.disconnect()

#### 2 Get incomming tick data 

In [7]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=15)

<IB connected to 127.0.0.1:7497 clientId=15>

In [8]:
contracts = [Forex(pair) for pair in ('EURUSD', 'USDJPY', 'GBPUSD', 'USDCHF', 'USDCAD', 'AUDUSD')]
ib.qualifyContracts(*contracts)

eurusd = contracts[0]

In [10]:
for contract in contracts:
    ib.reqMktData(contract, '', False, False)

In [11]:
ticker = ib.ticker(eurusd)
ib.sleep(2)

ticker

Ticker(contract=Forex('EURUSD', conId=12087792, exchange='IDEALPRO', localSymbol='EUR.USD', tradingClass='EUR.USD'), time=datetime.datetime(2023, 3, 6, 21, 21, 20, 206855, tzinfo=datetime.timezone.utc), minTick=1e-05, bid=1.06784, bidSize=1000000.0, ask=1.06786, askSize=2000000.0, prevBid=1.06785, prevBidSize=2000000.0, prevAskSize=1000000.0, high=1.06945, low=1.0622, close=1.0635, halted=0.0)

In [29]:
ticker.marketPrice()

1.0678649999999998

In [32]:
from IPython.display import display, clear_output
import pandas as pd

df = pd.DataFrame(
    index=[c.pair() for c in contracts],
    columns=['bidSize', 'bid', 'ask', 'askSize', 'high', 'low', 'close'])

def onPendingTickers(tickers):
    for t in tickers:
        df.loc[t.contract.pair()] = (
            t.bidSize, t.bid, t.ask, t.askSize, t.high, t.low, t.close)
        clear_output(wait=True)
    display(df)        

ib.pendingTickersEvent += onPendingTickers
ib.sleep(5)
ib.pendingTickersEvent -= onPendingTickers

,bidSize,bid,ask,askSize,high,low,close
EURUSD,2000000.0,1.06782,1.06783,1000000.0,1.06945,1.0622,1.0635
USDJPY,3000000.0,135.949,135.952,1500000.0,136.195,135.365,135.84
GBPUSD,3000000.0,1.20208,1.20213,1000000.0,1.2049,1.1993,1.2044
USDCHF,NaN,NaN,NaN,NaN,NaN,NaN,NaN
USDCAD,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AUDUSD,3000000.0,0.67303,0.67305,47000.0,0.67695,0.67165,0.6769


In [ ]:
for contract in contracts:
    ib.cancelMktData(contract)

In [ ]:
import asyncio

import ib_insync as ibi


class App:

    async def run(self):
        self.ib = ibi.IB()
        with await self.ib.connectAsync():
            contracts = [
                ibi.Stock(symbol, 'SMART', 'USD')
                for symbol in ['AAPL', 'TSLA', 'AMD', 'INTC']]
            for contract in contracts:
                self.ib.reqMktData(contract)

            async for tickers in self.ib.pendingTickersEvent:
                for ticker in tickers:
                    print(ticker)

    def stop(self):
        self.ib.disconnect()


app = App()
try:
    asyncio.run(app.run())
except (KeyboardInterrupt, SystemExit):
    app.stop()

#### 3 Basic Open/Close logic, position sizing

#### 4 Put order (market, limit)

In [40]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=16)
# util.logToConsole()

<IB connected to 127.0.0.1:7497 clientId=16>

In [ ]:
contract = Forex('EURUSD')
ib.qualifyContracts(contract)

order = LimitOrder('SELL', 20000, 1.11)

In [ ]:
trade = ib.placeOrder(contract, order)

In [ ]:
trade.log

In [ ]:
assert trade in ib.trades()

In [ ]:
# limit order
limitOrder = LimitOrder('BUY', 20000, 0.05)
limitTrade = ib.placeOrder(contract, limitOrder)

limitTrade

In [36]:
# Check order status
ib.sleep(1)
assert limitTrade.orderStatus.status == 'Submitted'

NameError: name 'limitTrade' is not defined

In [41]:
# Market Order

order = MarketOrder('BUY', 100)

trade = ib.placeOrder(contract, order)
while not trade.isDone():
    ib.waitOnUpdate()

In [42]:
contract

Forex('AUDUSD', conId=14433401, exchange='IDEALPRO', localSymbol='AUD.USD', tradingClass='AUD.USD')

In [ ]:
ib.positions()

In [ ]:
ib.disconnect()

In [44]:
###
from ib_insync import *

# Connect to the Interactive Brokers API
util.startLoop()
ib = IB()
ib.connect('127.0.0.1', 7497, clientId=0)

# Define the contract for the stock you want to trade
contract = Stock('AAPL', 'SMART', 'USD')

# Define the order parameters for a long position with leverage
order = LimitOrder('BUY', 100, 150.0, account='DU123456', leverage=4)

# Place the order
ib.placeOrder(contract, order)

# Disconnect from the API
ib.disconnect()



TypeError: Order.__init__() got an unexpected keyword argument 'leverage'

#### 5 Cancel open orders (end of the day)

In [ ]:
# Cancelar la orden a partir del id
ib.cancelOrder(limitOrder)
    
    

#### 6. Run ETL

In [ ]:
test = BaseModel()

In [ ]:
test.connect_to_ib()

In [ ]:
#### 7. Historical Data

In [ ]:

from ibapi.client import EClient
from ibapi.wrapper import EWrapper
from ibapi.contract import Contract

import threading
import time

class IBapi(EWrapper, EClient):
	def __init__(self):
		EClient.__init__(self, self)
		self.data = [] #Initialize variable to store candle

	def historicalData(self, reqId, bar):
		print(f'Time: {bar.date} Close: {bar.close}')
		self.data.append([bar.date, bar.close])
		
def run_loop():
	app.run()

app = IBapi()
app.connect('127.0.0.1', 7497, 123)

#Start the socket in a thread
api_thread = threading.Thread(target=run_loop, daemon=True)
api_thread.start()

time.sleep(1) #Sleep interval to allow time for connection to server

#Create contract object
eurusd_contract = Contract()
eurusd_contract.symbol = 'EUR'
eurusd_contract.secType = 'CASH'
eurusd_contract.exchange = 'IDEALPRO'
eurusd_contract.currency = 'USD'

#Request historical candles
app.reqHistoricalData(1, eurusd_contract, '', '2 D', '1 hour', 'BID', 0, 2, False, [])

time.sleep(5) #sleep to allow enough time for data to be returned

#Working with Pandas DataFrames
import pandas

df = pandas.DataFrame(app.data, columns=['DateTime', 'Close'])
df['DateTime'] = pandas.to_datetime(df['DateTime'],unit='s') 
df.to_csv('EURUSD_Hourly.csv')  

print(df)


app.disconnect()

In [33]:
import datetime

start = ''
end = datetime.datetime.now()
ticks = ib.reqHistoricalTicks(eurusd, start, end, 1000, 'BID_ASK', useRth=False)

ticks[-1]


HistoricalTickBidAsk(time=datetime.datetime(2023, 3, 6, 21, 26, 29, tzinfo=datetime.timezone.utc), tickAttribBidAsk=TickAttribBidAsk(bidPastLow=False, askPastHigh=False), priceBid=1.06792, priceAsk=1.06793, sizeBid=1000000.0, sizeAsk=2000000.0)

In [35]:
ib.disconnect()

In [ ]:
## Disconecting
ib.disconnect()